In [7]:
import awswrangler as wr
import pandas as pd
import os

In [ ]:
def split_csv(input_path: str, numerator: int, denominator: int) -> str:
    df = pd.read_csv(input_path)

    total_length = len(df)
    print(total_length)

    chunk_size = total_length // denominator
    print(chunk_size)


    start_row = (numerator-1) * chunk_size
    end_row = start_row + chunk_size

    print(start_row, end_row)


    df = df.iloc[start_row:end_row]
    df.to_csv("test.csv", index=False)

    return "test.csv"

In [ ]:
split_csv("../reference_library_10.csv", 2, 2)

In [2]:
df = pd.read_csv("../output.csv")

In [3]:
df

,key,input,hdac1_inh_pIC50_7,hdac1_inh_pIC50_8
0,PCQFQFRJSWBMEL-UHFFFAOYSA-N,COC(=O)C1=CC=CC2=C1C(=O)C1=CC([N+](=O)[O-])=CC...,0.015329,0.001902
1,MRSBJIAZTHGJAP-UHFFFAOYSA-N,CN(C)CCC1=CN(C)C2=CC=C(O)C=C12,0.002920,0.006755
2,CJUOVTMTGQENNQ-UHFFFAOYSA-N,CC1=C(S(=O)(=O)N2CCCCC2)C2=C(S1)N=CN(CC(=O)N1C...,0.037461,0.008910
3,OFCIHDDKDVHWGO-LICLKQGHSA-N,CN(C)CCOC1=CC=C(C(=O)/C=C/C2=CC=C(OC3=CC=CC=C3...,0.003356,0.000946
4,SZROQWMXFIMBGE-UHFFFAOYSA-N,O=C(CCC1=COC2=CC=CC(OCC3CCCCC3)=C2C1=O)C1=CC=C...,0.083075,0.079745


In [4]:
model_id = "eos2zmb"

In [5]:
columns = df.columns[-2:]

output = df[columns].to_dict("records")

df_postprocessed = df.copy()
df_postprocessed["output"] = output
df_postprocessed["model_id"] = model_id
df_postprocessed = df_postprocessed[["key", "input", "output", "model_id"]]
df_postprocessed.rename(columns={"key": "input_key", "input": "smiles"}, inplace=True)

In [6]:
df_postprocessed

,input_key,smiles,output,model_id
0,PCQFQFRJSWBMEL-UHFFFAOYSA-N,COC(=O)C1=CC=CC2=C1C(=O)C1=CC([N+](=O)[O-])=CC...,"{'hdac1_inh_pIC50_7': 0.015329241, 'hdac1_inh_...",eos2zmb
1,MRSBJIAZTHGJAP-UHFFFAOYSA-N,CN(C)CCC1=CN(C)C2=CC=C(O)C=C12,"{'hdac1_inh_pIC50_7': 0.00292032, 'hdac1_inh_p...",eos2zmb
2,CJUOVTMTGQENNQ-UHFFFAOYSA-N,CC1=C(S(=O)(=O)N2CCCCC2)C2=C(S1)N=CN(CC(=O)N1C...,"{'hdac1_inh_pIC50_7': 0.03746149, 'hdac1_inh_p...",eos2zmb
3,OFCIHDDKDVHWGO-LICLKQGHSA-N,CN(C)CCOC1=CC=C(C(=O)/C=C/C2=CC=C(OC3=CC=CC=C3...,"{'hdac1_inh_pIC50_7': 0.0033564952, 'hdac1_inh...",eos2zmb
4,SZROQWMXFIMBGE-UHFFFAOYSA-N,O=C(CCC1=COC2=CC=CC(OCC3CCCCC3)=C2C1=O)C1=CC=C...,"{'hdac1_inh_pIC50_7': 0.08307522, 'hdac1_inh_p...",eos2zmb


In [8]:
wr.s3.to_parquet(
    df=df_postprocessed,
    path=os.path.join(
        "s3://",
        "precalculations-bucket",
        "predictions",
    ),
    dataset=True,
    database="precalcs_test",
    table="predictions",
    partition_cols=["model_id"],
)

{'paths': ['s3://precalculations-bucket/predictions/model_id=eos2zmb/9c9cd8f64c934f5cb819463138cfc8e7.snappy.parquet'],
 'partitions_values': {'s3://precalculations-bucket/predictions/model_id=eos2zmb/': ['eos2zmb']}}